In [42]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import gudhi
import pickle
import numpy as np
import gudhi as gd
from LISM_simplicial import *


## Load data

In [44]:
with open('activation_data_mnist/simplexTree.pickle', 'rb') as handle:
    simplex_list = pickle.load(handle)

ST = gd.SimplexTree()
initial_value = -1e10

for simplex in simplex_list:
    ST.insert(simplex, initial_value)


In [45]:
stbase = gd.SimplexTree()
for i in range(len(simplex_list)):
    stbase.insert(np.array(simplex_list[i]), -1e10)
f = open('data/adv_simplextree.txt', 'w')
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write('\n')
f.close()

convention label : 0 normal, 1 fgsm, 2 fgsm noise, 3 cw, 4 cw noise

In [46]:
fgsm = np.load('activation_data_mnist/fgsm.npy')
fgsm_noise = np.load('activation_data_mnist/fgsm_noise.npy')
normal = np.load('activation_data_mnist/normal.npy')
cw = np.load('activation_data_mnist/cw.npy')
cw_noise = np.load('activation_data_mnist/cw_noise.npy')

## Compute distance matrix with possible data leakage

In [47]:
n_inputs = 20
features = [0,1]
data = np.zeros((n_inputs * 5, 260, 2))
labels = []
for i in range(5):
    labels += [i]*n_inputs


In [48]:
data[0:n_inputs, : ,: ] = fgsm[0:n_inputs, : , features ]
data[n_inputs:2*n_inputs, : ,: ] = fgsm_noise[0:n_inputs, : , features ]
data[2*n_inputs:3*n_inputs, : ,: ] = normal[0:n_inputs, : , features ]
data[3*n_inputs:4*n_inputs, : ,: ] = cw[0:n_inputs, : , features ]
data[4*n_inputs:5*n_inputs, : ,: ] = cw_noise[0:n_inputs, : , features ]


In [49]:
data.shape

(100, 260, 2)

In [50]:
distance_matrix = adversarial_pipeline("data/adv_simplextree.txt", data)

 53%|█████▎    | 27/51 [00:01<00:01, 22.19it/s]


KeyboardInterrupt: 

In [17]:
type(ST)

gudhi.simplex_tree.SimplexTree